# Environment Setup
Load packages & define parameters

In [101]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', None) #increase column width to unlimited
pd.set_option('display.max_rows', 50)
# !pip install yfinance
import yfinance as yf
from datetime import datetime
from yahoo_fin import stock_info as si
# Get market cap from here: https://api.tiingo.com/documentation/fundamentals

apikey = ''
url = "https://www.alphavantage.co/query"
stock = 'NNDM'

# Company Financial information
Summary figures from most recent filing

In [119]:
# Company Financial information 
# source: https://www.alphavantage.co/documentation/
# alternate source: Scrape FinViz - https://towardsdatascience.com/get-up-to-date-financial-ratios-p-e-p-b-and-more-of-stocks-using-python-4b53dd82908f

querystring = {"function":"OVERVIEW","symbol":stock,"apikey":apikey}
response = requests.request("GET", url, params=querystring)
# print(response.text)
response_tbl = pd.json_normalize(response.json())
(response_tbl)

#ALTERNATIVE CODE
# response = requests.get('https://www.alphavantage.co/query',
#     params={"function":"OVERVIEW","symbol":"TSLA","apikey":apikey}
# #     headers={'Authorization': '', 'Accept': 'application/json'}
# )
# print(response)
# json_response = response.json()
# print(response.status_code)
# print(json_response)

,Symbol,AssetType,Name,Description,Exchange,Currency,Country,Sector,Industry,Address,FullTimeEmployees,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,RevenuePerShareTTM,ProfitMargin,OperatingMarginTTM,ReturnOnAssetsTTM,ReturnOnEquityTTM,RevenueTTM,GrossProfitTTM,DilutedEPSTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,AnalystTargetPrice,TrailingPE,ForwardPE,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,SharesFloat,SharesShort,SharesShortPriorMonth,ShortRatio,ShortPercentOutstanding,ShortPercentFloat,PercentInsiders,PercentInstitutions,ForwardAnnualDividendRate,ForwardAnnualDividendYield,PayoutRatio,DividendDate,ExDividendDate,LastSplitFactor,LastSplitDate
0,NNDM,Common Stock,Nano Dimension Ltd,"Nano Dimension Ltd., together with its subsidi...",NASDAQ,USD,USA,Technology,Computer Hardware,"Ness-Ziona Science Park, Ness Ziona, Israel, 7...",68,December,2020-09-30,3271164160,-14278000,None,0,1.145,None,0,-0.45,0.155,0,-8.8981,-0.4209,-1.0198,3405000,2758000,-1.476,0,-0.805,10,0,0,195.8673,46.9995,716.875,0,2.5651,17.89,0.51,12.3515,5.6456,209690000,45019090,14067171,16638761,0.36,0.07,0,0,6.507,0,0,0,None,None,1:10,2019-10-22


#  Balance Sheet
Includes Annual & Quarterly financial filings

In [120]:
# Balance Sheet - This API returns the annual and quarterly balance sheets 
# for the company of interest. Data is generally refreshed on the same day a
# company reports its latest earnings and financials.

querystring = {"function":"BALANCE_SHEET",
               "symbol":stock,"apikey":apikey}
response = requests.request("GET", url, params=querystring)
Balance_Sheet_qtr = response.json()['quarterlyReports']
Balance_Sheet_qtr = pd.json_normalize(Balance_Sheet_qtr)
Balance_Sheet_qtr.insert(0,'Stock',stock)
# Balance_Sheet_qtr['Stock'] = stock #alternative

# Balance_Sheet_total = [annual,quarterly]
# Balance_Sheet = pd.concat(Balance_Sheet)

# Income Statement
Includes Annual & Quarterly financial filings

In [104]:
# Income Statement - This API returns the annual and quarterly income 
# statements for the company of interest. Data is generally refreshed on 
# the same day a company reports its latest earnings and financials.

querystring = {"function":"INCOME_STATEMENT",
               "symbol":stock,"apikey":apikey}
response = requests.request("GET", url, params=querystring)
Income_Statement_qtr = response.json()['quarterlyReports']
Income_Statement_qtr = pd.json_normalize(Income_Statement_qtr)
Income_Statement_qtr.insert(0,'Stock',stock)


In [105]:
# stock price today
common_stock_qtr = int(Balance_Sheet_qtr['commonStock'][0])
date= datetime.today() #date+' 11:00' #stock price on Qtr End Date
stock_price_qtr=si.get_live_price(stock)

## stock price on Qtr End Date
# date = (Income_Statement_qtr['fiscalDateEnding'][0]) #stock price on Qtr End Date
# datetime_object = datetime.strptime(date, '%Y-%m-%d %H:%M') 
# stock_price_qtr=yf.download(stock, start=date, end=date)['Adj Close'] #stock price on Qtr End Date

# Key Ratios


In [106]:
# Quarterly
workcap_qtr = (int(Balance_Sheet_qtr['totalCurrentAssets'][0])-int(Balance_Sheet_qtr['totalCurrentLiabilities'][0]))
total_assets_qtr = int(Balance_Sheet_qtr['totalAssets'][0])
retained_earnings_qtr = int(Balance_Sheet_qtr['retainedEarnings'][0])
total_liabilities_qtr = int(Balance_Sheet_qtr['totalLiabilities'][0])
mkt_capt_qtr = common_stock_qtr*stock_price_qtr
sales_qtr = Income_Statement_qtr['totalRevenue'][:4].astype(int).sum()
EBIT_qtr = Income_Statement_qtr['ebit'][:4].astype(int).sum()

# Altman Z Score
Assumption: This formula is basically designed for publicly held manufacturing firms with values of more than $1 million of net worth.


In [107]:
# Altman’s Z-Score: A numerical measurement used to predict the chances of a business going bankrupt in the next two years
# source: https://corporatefinanceinstitute.com/resources/knowledge/credit/altmans-z-score-model/
# The Z-score model is based on five key financial ratios, and it relies on the information contained in the 10-K report. 
# It increases the model’s accuracy when measuring the financial health of a company and its probability of going bankrupt.

# ζ = 1.2A + 1.4B + 3.3C + 0.6D + 1.0E 
# Where: Zeta (ζ) is the Altman’s Z-score 
#     A is the Working Capital/Total Assets ratio - Firm Liquidity
#     B is the Retained Earnings/Total Assets ratio - Profits compared to Assets
#     C is the Earnings Before Interest and Tax/Total Assets ratio - Profits generated by Assets
#     D is the Market Value of Equity/Total Liabilities ratio - Value versus Liabilities
#     E is the Total Sales/Total Assets ratio - Asset Efficiency Ratio

# Financial Distress - Zscore < 1.8; 
# Moderate Bankruptcy Risk - 1.8 < Zscore < 3; 
# Safe Zone - Zscore > 3
A = workcap_qtr / total_assets_qtr
B = retained_earnings_qtr / total_assets_qtr
C = EBIT_qtr / total_assets_qtr
D = mkt_capt_qtr / total_liabilities_qtr
E = sales_qtr / total_assets_qtr

Z = 1.2*A + 1.4*B + 3.3*C + 0.6*D + 1.0*E
Z
# print("A = {:.3n} B = {} C = {:.3n} D = {:.3n} E = {:.3n}".format(A,B,C,D,E))

55.45620456788436

# Altman Z Score
Assumption: This formula is modified for the case of private firms

In [108]:
# Altman’s Z-Score: A numerical measurement used to predict the chances of a business going bankrupt in the next two years
# source: https://corporatefinanceinstitute.com/resources/knowledge/credit/altmans-z-score-model/
# The Z-score model is based on five key financial ratios, and it relies on the information contained in the 10-K report. 
# It increases the model’s accuracy when measuring the financial health of a company and its probability of going bankrupt.

# ζ = 0.717A + 0.847B + 3.107C + 0.420D + 0.998E 
# Where: Zeta (ζ) is the Altman’s Z-score 
#     A is the Working Capital/Total Assets ratio 
#     B is the Retained Earnings/Total Assets ratio 
#     C is the Earnings Before Interest and Tax/Total Assets ratio 
#     D is the Book Value of Equity/Total Liabilities ratio 
#     E is the Total Sales/Total Assets ratio

# Financial Distress - Zscore < 1.23; 
# Moderate Bankruptcy Risk - 1.23 < Zscore < 3; 
# Safe Zone - Zscore > 3
A = workcap_qtr / total_assets_qtr
B = retained_earnings_qtr / total_assets_qtr
C = EBIT_qtr / total_assets_qtr
D = mkt_capt_qtr / total_liabilities_qtr
E = sales_qtr / total_assets_qtr

Z = 0.717*A + 0.847*B + 3.107*C + 0.42*D + 0.998*E
Z

38.4873104627168

# Altman Z Score
Assumption: This formula is modified for the case of non-manufacturing firms operating in emerging and developed markets.

In [109]:
# Altman’s Z-Score: A numerical measurement used to predict the chances of a business going bankrupt in the next two years
# source: https://corporatefinanceinstitute.com/resources/knowledge/credit/altmans-z-score-model/
# The Z-score model is based on five key financial ratios, and it relies on the information contained in the 10-K report. 
# It increases the model’s accuracy when measuring the financial health of a company and its probability of going bankrupt.

# Formula
# Non-manufacturing firm, operating in developed markets
# ζ = 6.56A + 3.26B + 6.72C + 1.05D 
# Non-manufacturing firm operating in emerging markets
# ζ = 3.25 + 6.56A + 3.26B + 6.72C + 1.05D 

# Where: Zeta (ζ) is the Altman’s Z-score 
#     A is the Working Capital/Total Assets ratio 
#     B is the Retained Earnings/Total Assets ratio 
#     C is the Earnings Before Interest and Tax/Total Assets ratio 
#     D is the Book Value of Equity/Total Liabilities ratio 

# Financial Distress - Zscore < 1.1; 
# Moderate Bankruptcy Risk - 1.1 < Zscore < 2.6; 
# Safe Zone - Zscore > 2.6
A = workcap_qtr / total_assets_qtr
B = retained_earnings_qtr / total_assets_qtr
C = EBIT_qtr / total_assets_qtr
D = mkt_capt_qtr / total_liabilities_qtr
E = sales_qtr / total_assets_qtr

Z = 6.65*A + 3.26*B + 6.72*C + 1.05*D
Z

97.92369182239437

# Cash Flow Statement
Includes Annual & Quarterly financial filings

In [122]:
# Cash Flow - This API returns the annual and quarterly cash flows 
# for the company of interest. Data is generally refreshed on the same day a 
# company reports its latest earnings and financials.

querystring = {"function":"CASH_FLOW",
               "symbol":stock,"apikey":apikey}
response = requests.request("GET", url, params=querystring)
Cashflow_ann = response.json()['annualReports']
Cashflow_ann = pd.json_normalize(Cashflow_ann)
Cashflow_ann.insert(0,'Stock',stock)

Cashflow_qtr = response.json()['quarterlyReports']
Cashflow_qtr = pd.json_normalize(Cashflow_qtr)
Cashflow_qtr.insert(0,'Stock',stock)

In [121]:
# Earnings - This API returns the annual and quarterly earnings (EPS) 
# for the company of interest. Quarterly data also includes analyst 
# estimates and surprise metrics.

querystring = {"function":"EARNINGS",
               "symbol":stock,"apikey":apikey}
response = requests.request("GET", url, params=querystring)
Earnings_ann = response.json()['annualEarnings']
Earnings_ann = pd.json_normalize(Earnings_ann)
Earnings_ann.insert(0,'Stock',stock)

Earnings_qtr = response.json()['quarterlyEarnings']
Earnings_qtr = pd.json_normalize(Earnings_qtr)
Earnings_qtr.insert(0,'Stock',stock)


In [ ]:
# Listing & Delisting Status - This API returns a list of active or delisted
# US stocks and ETFs, either as of the latest trading day or at a specific 
# time in history. The endpoint is positioned to facilitate equity research
# on asset lifecycle and survivorship.
from io import StringIO

#delisted stocks as of the latest trading day
querystring = {"function":"LISTING_STATUS","state":"delisted","apikey":apikey}
response = requests.request("GET", url, params=querystring)
delisted=pd.read_csv(StringIO(response.text))

#active stocks as of the latest trading day
querystring = {"function":"LISTING_STATUS","state":"active","apikey":apikey}
response = requests.request("GET", url, params=querystring)
active=pd.read_csv(StringIO(response.text))